# 🌀 Cyclone Evacuation Simulation

This notebook demonstrates a **disaster evacuation simulation** where civilians escape from an approaching cyclone.

## Key Features:
- **Moving Cyclone**: Approaches from right to left with vertical wobble
- **Danger Field**: Gaussian distribution showing danger intensity
- **Wind Field**: Point Vortex model creating spinning wind patterns
- **Civilian AI**: Uses gradient descent + pathfinding to reach safe zones

## Physics Models Used:
1. **Gaussian Function** for danger field
2. **Point Vortex Model** for wind field
3. **Gradient Descent** for evacuation pathfinding

## Civilian Velocity Equation:
$$\vec{V} = 1.8 \times \vec{V}_{away} + 0.25 \times \vec{V}_{wind} + 0.4 \times \vec{V}_{safety}$$

## 1. Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Enable interactive plots in Jupyter
%matplotlib inline

## 2. Define Grid Parameters

Create a **160×160 grid** spanning from -2 to +2 in both X and Y directions.
- `X, Y` are 2D matrices where each point has coordinates
- This is where we calculate danger and wind values

In [ ]:
# Grid resolution
nx, ny = 160, 160

# Create 1D arrays for x and y coordinates
x = np.linspace(-2, 2, nx)  # 160 points from -2 to 2
y = np.linspace(-2, 2, ny)  # 160 points from -2 to 2

# Create 2D coordinate matrices
X, Y = np.meshgrid(x, y)

print(f"Grid shape: {X.shape}")
print(f"X range: [{X.min()}, {X.max()}]")
print(f"Y range: [{Y.min()}, {Y.max()}]")

## 3. Define Cyclone Movement

The cyclone moves from **right to left** across the map:
- Starts at x = 2.5 (off-screen right)
- Moves at 0.02 units per frame
- Has sinusoidal vertical wobble (amplitude = 0.6)

In [ ]:
def cyclone_center(t):
    """
    Calculate cyclone center position at time t.
    
    Returns:
        cx: X-coordinate (moves right to left)
        cy: Y-coordinate (wobbles up and down)
    """
    cx = 2.5 - 0.02 * t       # moves from x = +2.5 → -2.5
    cy = 0.6 * np.sin(0.1*t)  # slight vertical wobble
    return cx, cy

# Test the function
for t in [0, 50, 100, 150]:
    cx, cy = cyclone_center(t)
    print(f"t={t:3d}: cyclone at ({cx:.2f}, {cy:.2f})")

## 4. Define Safe Points and Civilians

**Safe Points**: Corners where civilians can escape (marked in green)

**Civilians**: 30 agents with:
- Random starting positions between (-1, 1)
- Velocity vectors (for movement arrows)
- Arrival status tracking

In [ ]:
# Safe corners (left side - away from cyclone coming from right)
safe_points = np.array([
    [x[0], y[-1]],     # top-left corner (-2, 2)
    [x[0], y[0]],      # bottom-left corner (-2, -2)
])

# Civilians setup
num_civ = 30
civ_pos = np.random.uniform(-1.0, 1.0, (num_civ, 2))  # random positions
civ_vel = np.zeros_like(civ_pos)                       # velocity vectors
arrived = np.zeros(num_civ, dtype=bool)                # track who reached safety

print(f"Safe points: {safe_points}")
print(f"Number of civilians: {num_civ}")
print(f"Starting positions shape: {civ_pos.shape}")

## 5. Danger Field (Gaussian Distribution)

The danger field uses a **Gaussian (bell curve)** function:

$$D(x,y) = e^{-\frac{(x-c_x)^2 + (y-c_y)^2}{R^2}}$$

- **Center**: Maximum danger (D = 1) at cyclone center
- **Edge**: Danger decreases exponentially with distance
- **R = 3.0**: Controls the radius of the danger zone

In [ ]:
def danger_field(cx, cy):
    """
    Calculate danger intensity at every grid point.
    Uses Gaussian function centered on cyclone.
    
    Args:
        cx, cy: Cyclone center coordinates
    Returns:
        2D array of danger values (0 to 1)
    """
    R = 3.0  # Cyclone radius
    return np.exp(-((X - cx)**2 + (Y - cy)**2) / (R**2))

# Visualize the danger field
cx0, cy0 = cyclone_center(0)
D0 = danger_field(cx0, cy0)

plt.figure(figsize=(8, 6))
plt.imshow(D0, extent=(-2, 2, -2, 2), origin='lower', cmap='inferno')
plt.colorbar(label='Danger Level')
plt.scatter([cx0], [cy0], c='cyan', s=100, marker='x', label='Cyclone Center')
plt.title('Danger Field (Gaussian Distribution)')
plt.xlabel('X'); plt.ylabel('Y')
plt.legend()
plt.show()

## 6. Gradient Calculation

The **gradient** tells us the direction of steepest increase in danger.
Civilians move in the **negative gradient** direction (downhill = away from danger).

$$\nabla D = \left( \frac{\partial D}{\partial x}, \frac{\partial D}{\partial y} \right)$$

In [ ]:
def grad(F):
    """
    Calculate gradient of a 2D scalar field.
    
    Args:
        F: 2D array (danger field)
    Returns:
        Fx: Gradient in X direction
        Fy: Gradient in Y direction
    """
    Fx = np.gradient(F, axis=1)  # x direction (columns)
    Fy = np.gradient(F, axis=0)  # y direction (rows)
    return Fx, Fy

# Visualize gradient
Dx, Dy = grad(D0)
stride = 10

plt.figure(figsize=(8, 6))
plt.imshow(D0, extent=(-2, 2, -2, 2), origin='lower', cmap='inferno', alpha=0.7)
plt.quiver(X[::stride, ::stride], Y[::stride, ::stride],
           -Dx[::stride, ::stride], -Dy[::stride, ::stride],  # Negative = away from danger
           color='white', alpha=0.8)
plt.title('Negative Gradient (Escape Direction)')
plt.xlabel('X'); plt.ylabel('Y')
plt.show()

## 7. Wind Field (Point Vortex Model)

The wind field creates a **spinning pattern** around the cyclone using the **Point Vortex Model**:

$$u = -\frac{Y_c}{r^2} \quad \text{(horizontal wind)}$$
$$v = +\frac{X_c}{r^2} \quad \text{(vertical wind)}$$

Where $r = \sqrt{X_c^2 + Y_c^2}$ is distance from cyclone center.

In [ ]:
def wind_field(cx, cy):
    """
    Calculate wind velocity at every grid point.
    Uses Point Vortex Model (counter-clockwise rotation).
    
    Args:
        cx, cy: Cyclone center coordinates
    Returns:
        u: Horizontal wind velocity (2D array)
        v: Vertical wind velocity (2D array)
    """
    Xc = X - cx                          # Distance from center (X)
    Yc = Y - cy                          # Distance from center (Y)
    r = np.sqrt(Xc**2 + Yc**2) + 1e-9    # Radius (avoid /0)
    strength = 1.0
    u = -strength * (Yc / (r**2))        # Horizontal wind
    v =  strength * (Xc / (r**2))        # Vertical wind
    return u, v

# Visualize wind field
U0, V0 = wind_field(cx0, cy0)
stride = 8

plt.figure(figsize=(8, 6))
plt.imshow(D0, extent=(-2, 2, -2, 2), origin='lower', cmap='inferno', alpha=0.5)
plt.quiver(X[::stride, ::stride], Y[::stride, ::stride],
           U0[::stride, ::stride], V0[::stride, ::stride],
           color='cyan', alpha=0.9)
plt.scatter([cx0], [cy0], c='white', s=100, marker='x')
plt.title('Wind Field (Point Vortex - Counter-clockwise)')
plt.xlabel('X'); plt.ylabel('Y')
plt.show()

## 8. Direction to Safety Function

This function finds the **nearest safe point** and returns:
1. Normalized direction vector (unit vector)
2. Distance to nearest safe point

In [ ]:
def direction_to_safety(px, py):
    """
    Calculate direction to nearest safe point.
    
    Args:
        px, py: Civilian position
    Returns:
        dx/L, dy/L: Normalized direction vector (unit vector)
        distance: Distance to nearest safe point
    """
    # Calculate distance to each safe point
    d = np.sqrt((safe_points[:,0]-px)**2 + (safe_points[:,1]-py)**2)
    
    # Find nearest safe point
    idx = np.argmin(d)
    tx, ty = safe_points[idx]
    
    # Direction to safe point
    dx = tx - px
    dy = ty - py
    
    # Normalize (make unit vector)
    L = np.sqrt(dx*dx + dy*dy) + 1e-9
    
    return dx/L, dy/L, np.min(d)

# Test with a civilian at (0, 0)
sx, sy, dist = direction_to_safety(0, 0)
print(f"From (0,0): direction = ({sx:.3f}, {sy:.3f}), distance = {dist:.3f}")

## 9. Setup Matplotlib Figure

Create the visualization with:
- **Heat map**: Danger field (inferno colormap)
- **Cyan arrows**: Wind field vectors
- **Green dots**: Safe zones
- **White dots**: Civilians with velocity arrows

In [ ]:
# Reset civilians for animation
civ_pos = np.random.uniform(-1.0, 1.0, (num_civ, 2))
civ_vel = np.zeros_like(civ_pos)
arrived = np.zeros(num_civ, dtype=bool)

# Create figure
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xticks([]); ax.set_yticks([])

# Initial danger field
cx0, cy0 = cyclone_center(0)
D0 = danger_field(cx0, cy0)
im = ax.imshow(D0, extent=(x.min(), x.max(), y.min(), y.max()),
               origin="lower", cmap="inferno", alpha=0.8)

# Wind arrows
stride = 8
U0, V0 = wind_field(cx0, cy0)
quiv = ax.quiver(X[::stride,::stride], Y[::stride,::stride],
                 U0[::stride,::stride], V0[::stride,::stride],
                 color="cyan", alpha=0.8)

# Safe points (green)
ax.scatter(safe_points[:,0], safe_points[:,1],
           s=150, c="lime", edgecolors="black", label="Safe Zones", zorder=5)

# Civilians (white)
civ_scatter = ax.scatter(civ_pos[:,0], civ_pos[:,1],
                         c="white", s=50, edgecolors="black", label="Civilians", zorder=4)

# Civilian velocity arrows
civ_quiv = ax.quiver(civ_pos[:,0], civ_pos[:,1],
                     civ_vel[:,0], civ_vel[:,1],
                     color="white", scale=5, width=0.008)

ax.set_xlim(-2, 2); ax.set_ylim(-2, 2)
ax.legend(loc="upper right")
plt.close()  # Don't show static plot

## 10. Animation Update Function

Each frame:
1. Update cyclone position
2. Recalculate danger field and wind field
3. Move each civilian using the velocity equation:

$$\vec{V} = 1.8 \times \vec{V}_{away} + 0.25 \times \vec{V}_{wind} + 0.4 \times \vec{V}_{safety}$$

In [ ]:
def update(f):
    """Animation update function - called each frame."""
    global civ_pos, civ_vel, arrived

    t = f * 0.15                    # Convert frame to time
    cx, cy = cyclone_center(t)      # Get cyclone position

    # Update danger field
    D = danger_field(cx, cy)
    im.set_data(D)

    # Calculate gradient (escape direction)
    Dx, Dy = grad(D)

    # Update wind field
    U, V = wind_field(cx, cy)
    quiv.set_UVC(U[::stride,::stride], V[::stride,::stride])

    # Move each civilian
    for i in range(num_civ):
        if arrived[i]:
            civ_vel[i] = np.array([0.0, 0.0])
            continue

        px, py = civ_pos[i]
        
        # Find nearest grid point
        ix = np.argmin(np.abs(x - px))
        iy = np.argmin(np.abs(y - py))

        # 1. Move AWAY from cyclone (negative gradient)
        away = np.array([-Dx[iy,ix], -Dy[iy,ix]])

        # 2. Wind pushes them
        wind = np.array([U[iy,ix], V[iy,ix]])

        # 3. Move toward safety
        sx, sy, dist_safe = direction_to_safety(px, py)
        safety = np.array([sx, sy]) * 0.4

        # Total velocity: V = 1.8*away + 0.25*wind + safety
        vel = 1.8*away + 0.25*wind + safety
        civ_pos[i] += vel * 0.02    # Update position
        civ_vel[i] = vel             # Store for display

        # Check if reached safety
        if dist_safe < 0.05:
            arrived[i] = True
            civ_vel[i] = np.array([0.0, 0.0])

    # Update display
    civ_scatter.set_offsets(civ_pos)
    civ_quiv.set_offsets(civ_pos)
    civ_quiv.set_UVC(civ_vel[:,0], civ_vel[:,1])

    num_arrived = np.sum(arrived)
    ax.set_title(f"Cyclone Evacuation | t={t:.2f}s | Evacuated: {num_arrived}/{num_civ}")
    
    return im, quiv, civ_scatter, civ_quiv

## 11. Run Animation

Create and display the animation in Jupyter notebook.

In [ ]:
# Create animation
ani = FuncAnimation(fig, update, frames=200, interval=50, blit=False)

# Display in Jupyter notebook
HTML(ani.to_jshtml())

---
## References

1. **Pedestrian Evacuation Model**: Helbing, D., Farkas, I., & Vicsek, T. (2000). *Simulating dynamical features of escape panic.* Nature, 407, 487-490.
   - https://www.nature.com/articles/35035023

2. **Point Vortex Model**: https://en.wikipedia.org/wiki/Point_vortex

3. **Gaussian Function**: https://en.wikipedia.org/wiki/Gaussian_function

4. **Gradient Descent**: https://en.wikipedia.org/wiki/Gradient_descent

5. **NumPy Gradient**: https://numpy.org/doc/stable/reference/generated/numpy.gradient.html